# SA-CCR Euler allocation of an exemplary equity portfolio

We set up a collateral agreement with no thresholds or MTA and add two equity options to it.
A big put option on an imagainary stock ADS and a smaller call option on the same stock. Both options are struck at the current market price $S(0)$.

We consider three cases of margining

1. No margining
2. VM only
3. VM and bilateral IM

In [1]:
import QuantLib as ql
from IPython.core.display import display, Markdown, Image
from allocation.Enums import FdApproach2
from allocation.eulerAllocator import EulerAllocator
from collateralAgreement.collateralAgreement import CollateralAgreement, Margining, InitialMargining
from instruments.equity_instruments.equityOption import EquityOption
from jupyterUtils import export
from sa_ccr.sa_ccr import SA_CCR
from utilities.Enums import Stock, TradeType, TradeDirection
from marketdata import init_marketdata
import pandas as pd
asdf =1

In [2]:
ca = CollateralAgreement(threshold=0,
                         threshold_vm=0,
                         mta=0,
                         margining=Margining.UNMARGINED,
                         initialMargining=InitialMargining.NO_IM)
ca.link_sa_ccr_instance(SA_CCR(ca))

ca_vm = CollateralAgreement(threshold=0,
                            threshold_vm=0,
                            mta=0,
                            margining=Margining.MARGINED,
                            initialMargining=InitialMargining.NO_IM)
ca_vm.link_sa_ccr_instance(SA_CCR(ca_vm))

ca_im = CollateralAgreement(threshold=0,
                            threshold_vm=0,
                            mta=0,
                            margining=Margining.MARGINED,
                            initialMargining=InitialMargining.SIMM)
ca_im.link_sa_ccr_instance(SA_CCR(ca_im))

In [3]:
eq_opt_ads_call = EquityOption(underlying=Stock.ADS,
                               maturity=ql.Period(1, ql.Years),
                               notional=2000000,
                               tradeType=TradeType.CALL,
                               tradeDirection=TradeDirection.LONG)

eq_opt_ads_put = EquityOption(underlying=Stock.ADS,
                              maturity=ql.Period(1, ql.Years),
                              notional=3000000,
                              tradeType=TradeType.PUT,
                              tradeDirection=TradeDirection.LONG)

In [4]:
display(Markdown('The current value of these trades is {:,.0f} EUR for the call option and {:,.0f} EUR for the put option.'.format(eq_opt_ads_call.get_price(), eq_opt_ads_put.get_price())))

The current value of these trades is 6,601,467 EUR for the call option and 10,378,881 EUR for the put option.

When putting these trades in the three portfolios we observe a high difference between the unmargined and VM margined EAD. This difference is primarily driven through the RC and therefore through the high positive PV.

In [5]:
ca.add_trades([eq_opt_ads_call, eq_opt_ads_put])
ca_vm.add_trades([eq_opt_ads_call, eq_opt_ads_put])
ca_im.add_trades([eq_opt_ads_call,eq_opt_ads_put])

In [6]:
print('EAD with no margining: {:,.2f} USD'.format(ca.get_sa_ccr_model().get_ead()))
print('EAD with VM margining: {:,.2f} USD'.format(ca_vm.get_sa_ccr_model().get_ead()))
print('RC with no margining: {:,.2f} USD'.format(ca.get_sa_ccr_model().get_rc()))
print('RC with VM margining: {:,.2f} USD'.format(ca_vm.get_sa_ccr_model().get_rc()))
print('Portfolio PV: {:,.2f} USD'.format(ca.get_V()))


EAD with no margining: 37,643,536.02 USD
EAD with VM margining: 3,519,457.62 USD
RC with no margining: 18,508,579.01 USD
RC with VM margining: 0.00 USD
Portfolio PV: 18,508,579.01 USD


Next, we perform an Euler allocation:

In [7]:
eulerAllocator1 = EulerAllocator(ca)
eulerAllocator2 = EulerAllocator(ca_vm)
eulerAllocator3 = EulerAllocator(ca_im)
allocation_no_margin = eulerAllocator1.allocate_ead()
allocation_with_vm = eulerAllocator2.allocate_ead()
allocation_with_im_and_im = eulerAllocator3.allocate_ead()
allocation_im = eulerAllocator3.allocate_im()

we can calculate how far the sum of the allocated values deviates from the risk measure:

In [8]:
delta_no_margin = abs(ca.get_sa_ccr_model().get_risk_measure()-sum([allocation_no_margin[t] for t in ca.trades]))
delta_vm = abs(ca_vm.get_sa_ccr_model().get_risk_measure()-sum([allocation_with_vm[t] for t in ca_vm.trades]))
delta_im_calc = abs(ca_im.get_im_model().get_risk_measure()-sum([allocation_im[t] for t in ca_im.trades]))
delta_vm_and_im = abs(ca_im.get_sa_ccr_model().get_risk_measure()-sum([allocation_with_im_and_im[t] for t in ca_im.trades]))

In [9]:
print('Diff EAD no margin:    %.2f EUR' %(delta_no_margin))
print('Diff EAD only VM:      %.2f EUR' %(delta_vm))
print('Diff EAD VM + IM:   %.2f EUR' %(delta_vm_and_im))
print('Diff calculated IM:    %.2f EUR' %(delta_im_calc))

Diff EAD no margin:    0.00 EUR
Diff EAD only VM:      0.00 EUR
Diff EAD VM + IM:   1068.64 EUR
Diff calculated IM:    6.74 EUR


In [10]:
display(Markdown('In relation to the EAD of {:,.0f} EUR the deviation of the allocated EAD under VM and IM of {:,.2f} EUR is not large but can be improved nevertheless.'.format(ca_im.get_sa_ccr_model().get_risk_measure(), delta_vm_and_im)))

In relation to the EAD of 345,874 EUR the deviation of the allocated EAD under VM and IM of 1,068.64 EUR is not large but can be improved nevertheless.

By, default the implemented Euler allocation class uses a forward difference approach. If we switch over to a central difference approach the deviation shrinks significantly.

In [11]:
eulerAllocator3.fdApproach2 = FdApproach2.Central
allocation_im = eulerAllocator3.allocate_im()
allocation_with_im_and_im = eulerAllocator3.allocate_ead()

In [12]:
delta_vm_and_im = abs(ca_im.get_sa_ccr_model().get_risk_measure()-sum([allocation_with_im_and_im[t] for t in ca_im.trades]))
delta_im_calc = abs(ca_im.get_im_model().get_risk_measure()-sum([allocation_im[t] for t in ca_im.trades]))

In [13]:
print('Diff EAD VM + IM:   %.2f EUR' %(delta_vm_and_im))
print('Diff calculated IM: %.2f EUR' %(delta_im_calc))

Diff EAD VM + IM:   0.01 EUR
Diff calculated IM: 0.00 EUR


Displaying the allocation results

In [14]:
def trrep(trade):
    import ast
    dic = ast.literal_eval(str(trade))
    return '{:.0f}Mn '.format(dic['Notional']/1000000)+trade.underlying.name+' '+dic['TradeType']

simm_entry = pd.Series({'Title':'SIMM',
                         trrep(eq_opt_ads_call):allocation_im[eq_opt_ads_call]/ca_im.get_im_model().get_risk_measure(),
                         trrep(eq_opt_ads_put):allocation_im[eq_opt_ads_put]/ca_im.get_im_model().get_risk_measure(),
                         'Portfolio Risk Measure':ca_im.get_im_model().get_risk_measure()})

result_df = pd.DataFrame().append(simm_entry, ignore_index=True)

ead_no_margin_entry = pd.Series({'Title':'No margin',
                         trrep(eq_opt_ads_call):allocation_no_margin[eq_opt_ads_call]/ca.get_sa_ccr_model().get_risk_measure(),
                         trrep(eq_opt_ads_put):allocation_no_margin[eq_opt_ads_put]/ca.get_sa_ccr_model().get_risk_measure(),
                         'Portfolio Risk Measure':ca.get_sa_ccr_model().get_risk_measure()})

result_df = result_df.append(ead_no_margin_entry, ignore_index=True)


ead_vm_only_entry = pd.Series({'Title':'VM only',
                         trrep(eq_opt_ads_call):allocation_with_vm[eq_opt_ads_call]/ca_vm.get_sa_ccr_model().get_risk_measure(),
                         trrep(eq_opt_ads_put):allocation_with_vm[eq_opt_ads_put]/ca_vm.get_sa_ccr_model().get_risk_measure(),
                         'Portfolio Risk Measure':ca_vm.get_sa_ccr_model().get_risk_measure()})

result_df = result_df.append(ead_vm_only_entry, ignore_index=True)

ead_vm_im_entry = pd.Series({'Title':'VM+IM',
                         trrep(eq_opt_ads_call):allocation_with_im_and_im[eq_opt_ads_call]/ca_im.get_sa_ccr_model().get_risk_measure(),
                         trrep(eq_opt_ads_put):allocation_with_im_and_im[eq_opt_ads_put]/ca_im.get_sa_ccr_model().get_risk_measure(),
                         'Portfolio Risk Measure':ca_im.get_sa_ccr_model().get_risk_measure()})

result_df = result_df.append(ead_vm_im_entry, ignore_index=True)

result_df.set_index('Title', inplace=True)
result_df.index.name = None

result_df[trrep(eq_opt_ads_call)] = result_df[trrep(eq_opt_ads_call)].apply(lambda s:'{:.2%}'.format(s))
result_df[trrep(eq_opt_ads_put)] = result_df[trrep(eq_opt_ads_put)].apply(lambda s:'{:.2%}'.format(s))
result_df['Portfolio Risk Measure'] = result_df['Portfolio Risk Measure'].apply(lambda s:'{:,.0f} USD'.format(s))
result_df

,2Mn ADS Call,3Mn ADS Put,Portfolio Risk Measure
SIMM,-33.75%,133.75%,"14,231,564 USD"
No margin,99.21%,0.79%,"37,643,536 USD"
VM only,232.47%,-132.47%,"3,519,458 USD"
VM+IM,622.10%,-522.10%,"345,874 USD"


For SIMM the Put has the higher risk and the Call is considered a hedge trade while for SA-CCR with only VM, the Call has the higher risk and the Put is considered a hedge trade.

The reason for this are the different holding periods between the two models. If we lower the maturity of the trades to 10 days instead, we can see that for SA-CCR with only VM the call is considered the hedge trade.



In [15]:
eq_opt_ads_call_10d = EquityOption(underlying=Stock.ADS,
                               maturity=ql.Period(10, ql.Days),
                               notional=2000000,
                               tradeType=TradeType.CALL,
                               tradeDirection=TradeDirection.LONG)

eq_opt_ads_put_10d = EquityOption(underlying=Stock.ADS,
                              maturity=ql.Period(10, ql.Days),
                              notional=3000000,
                              tradeType=TradeType.PUT,
                              tradeDirection=TradeDirection.LONG)

In [16]:
ca_vm.remove_all_trades()
ca_vm.add_trades([eq_opt_ads_call_10d,eq_opt_ads_put_10d])

In [17]:
allocation_with_vm = eulerAllocator2.allocate_ead()

In [18]:
def trrep2(trade):
    import ast
    dic = ast.literal_eval(str(trade))
    return '{:.0f}Mn '.format(dic['Notional']/1000000)+trade.underlying.name+' '+dic['TradeType']+' ' +str(trade.ql_maturity)

In [19]:
ead_vm_only_entry = pd.Series({'Title':'VM only',
    trrep2(eq_opt_ads_call_10d):allocation_with_vm[eq_opt_ads_call_10d]/ca_vm.get_sa_ccr_model().get_risk_measure(),
    trrep2(eq_opt_ads_put_10d):allocation_with_vm[eq_opt_ads_put_10d]/ca_vm.get_sa_ccr_model().get_risk_measure(),
    'Portfolio Risk Measure':ca_vm.get_sa_ccr_model().get_risk_measure()})

result_df = pd.DataFrame().append(ead_vm_only_entry, ignore_index=True)

result_df.set_index('Title', inplace=True)
result_df.index.name = None

result_df[trrep2(eq_opt_ads_call_10d)] = result_df[trrep2(eq_opt_ads_call_10d)].apply(lambda s:'{:.2%}'.format(s))
result_df[trrep2(eq_opt_ads_put_10d)] = result_df[trrep2(eq_opt_ads_put_10d)].apply(lambda s:'{:.2%}'.format(s))
result_df['Portfolio Risk Measure'] = result_df['Portfolio Risk Measure'].apply(lambda s:'{:,.0f} USD'.format(s))
result_df

,2Mn ADS Call 1W3D,3Mn ADS Put 1W3D,Portfolio Risk Measure
VM only,-358.06%,458.06%,"1,701,707 USD"


Going back to 1Y maturity equity options we can see that the allocations of the preexisting trades can change significantly, when we add another equity option to the portfolio. We choose a position of 10Mn call options on an imaginary DBK stock.

In [20]:
eq_opt_dbk_call = EquityOption(underlying=Stock.DBK,
                               maturity=ql.Period(1, ql.Years),
                               notional = 10000000,
                               tradeType=TradeType.CALL,
                               tradeDirection=TradeDirection.LONG)

ca.remove_all_trades()
ca.add_trades([eq_opt_ads_call,eq_opt_ads_put,eq_opt_dbk_call])
ca_vm.remove_all_trades()
ca_vm.add_trades([eq_opt_ads_call,eq_opt_ads_put,eq_opt_dbk_call])
ca_im.remove_all_trades()
ca_im.add_trades([eq_opt_ads_call,eq_opt_ads_put,eq_opt_dbk_call])

In [21]:
allocation_no_margin = eulerAllocator1.allocate_ead()
allocation_with_vm = eulerAllocator2.allocate_ead()
allocation_with_im_and_im = eulerAllocator3.allocate_ead()
allocation_im = eulerAllocator3.allocate_im()

In [22]:
simm_entry = pd.Series({'Title':'SIMM',
    trrep(eq_opt_ads_call):allocation_im[eq_opt_ads_call]/ca_im.get_im_model().get_risk_measure(),
    trrep(eq_opt_ads_put):allocation_im[eq_opt_ads_put]/ca_im.get_im_model().get_risk_measure(),
    trrep(eq_opt_dbk_call):allocation_im[eq_opt_dbk_call]/ca_im.get_im_model().get_risk_measure(),
    'PF Risk Measure':ca_im.get_im_model().get_risk_measure()})

result_df = pd.DataFrame().append(simm_entry, ignore_index=True)

ead_no_margin_entry = pd.Series({'Title':'No margin',
    trrep(eq_opt_ads_call):allocation_no_margin[eq_opt_ads_call]/ca.get_sa_ccr_model().get_risk_measure(),
    trrep(eq_opt_ads_put):allocation_no_margin[eq_opt_ads_put]/ca.get_sa_ccr_model().get_risk_measure(),
    trrep(eq_opt_dbk_call):allocation_no_margin[eq_opt_dbk_call]/ca.get_sa_ccr_model().get_risk_measure(),
    'PF Risk Measure':ca.get_sa_ccr_model().get_risk_measure()})

result_df = result_df.append(ead_no_margin_entry, ignore_index=True)


ead_vm_only_entry = pd.Series({'Title':'VM only',
    trrep(eq_opt_ads_call):allocation_with_vm[eq_opt_ads_call]/ca_vm.get_sa_ccr_model().get_risk_measure(),
    trrep(eq_opt_ads_put):allocation_with_vm[eq_opt_ads_put]/ca_vm.get_sa_ccr_model().get_risk_measure(),
    trrep(eq_opt_dbk_call):allocation_with_vm[eq_opt_dbk_call]/ca_vm.get_sa_ccr_model().get_risk_measure(),
    'PF Risk Measure':ca_vm.get_sa_ccr_model().get_risk_measure()})

result_df = result_df.append(ead_vm_only_entry, ignore_index=True)

ead_vm_im_entry = pd.Series({'Title':'VM+IM',
    trrep(eq_opt_ads_call):allocation_with_im_and_im[eq_opt_ads_call]/ca_im.get_sa_ccr_model().get_risk_measure(),
    trrep(eq_opt_ads_put):allocation_with_im_and_im[eq_opt_ads_put]/ca_im.get_sa_ccr_model().get_risk_measure(),
    trrep(eq_opt_dbk_call):allocation_with_im_and_im[eq_opt_dbk_call]/ca_im.get_sa_ccr_model().get_risk_measure(),
    'PF Risk Measure':ca_im.get_sa_ccr_model().get_risk_measure()})

result_df = result_df.append(ead_vm_im_entry, ignore_index=True)

result_df.set_index('Title', inplace=True)
result_df.index.name = None
result_df[trrep(eq_opt_ads_call)] = result_df[trrep(eq_opt_ads_call)].apply(lambda s:'{:.2%}'.format(s))
result_df[trrep(eq_opt_ads_put)] = result_df[trrep(eq_opt_ads_put)].apply(lambda s:'{:.2%}'.format(s))
result_df[trrep(eq_opt_dbk_call)] = result_df[trrep(eq_opt_dbk_call)].apply(lambda s:'{:.2%}'.format(s))
result_df['PF Risk Measure'] = result_df['PF Risk Measure'].apply(lambda s:'{:,.0f} USD'.format(s))
result_df
#
# result_df.style.format({trrep(eq_opt_ads_call):'{:.2%}',
#                         trrep(eq_opt_ads_put):'{:.2%}',
#                         trrep(eq_opt_dbk_call):'{:.2%}',
#                         'Portfolio Risk Measure':'{:,.0f} USD'})

,10Mn DBK Call,2Mn ADS Call,3Mn ADS Put,PF Risk Measure
SIMM,63.10%,15.23%,21.67%,"27,551,513 USD"
No margin,57.45%,33.16%,9.39%,"76,295,560 USD"
VM only,80.79%,44.65%,-25.44%,"10,230,051 USD"
VM+IM,106.19%,86.85%,-93.04%,"1,847,365 USD"


Further analysis of the results shown above may be found in section \ref{sec:Exemplary Euler allocation of SA-CCR under consideration of margining}

In [26]:
export('SA-CCR Euler allocation of an exemplary equity portfolio.ipynb')